In [43]:
import random
import numpy as np
import pandas as pd

def get_manip(row):
    cond = row["all_cond"]
    if "dom" in cond:
        return "dominant"
    else:
        return "submissive"
    
def get_condition(row):
    cond = row["all_cond"]
    cond = cond.replace("_dom", "")
    cond = cond.replace("_sub", "")
    return cond
        
def clear_non_conflict_manip(row):
    trial_cond = row["trial_cond"]
    manip = row["manip"]

    if trial_cond == "NCE" or trial_cond == "NCH":
        manip = "no_manip"

    return manip


def get_n_roundConsecutives(df): 
    players = ["1", "2", "3", "4", "5", "6"]
    df["other_player_str"] = df["other_participant"].astype(str)
    df["participant_str"] = df["participant"].astype(str)

    distance_dict = {}

    for player in players:
        df_player     = df[df["participant_str"] == player]
        other_players = [p for p in players if p != player]

        distance_dict[player] = 0
        for other in other_players:
            
            last_seen = 0
            for index,row in df_player.iterrows():
                if other == row["other_player_str"]:
                    distance  = row["round_nb"] - last_seen
                    last_seen = row["round_nb"]
                    if distance == 1:
                        distance_dict[f"{player}"] += 1
    return distance_dict

In [150]:
extra_conflict = [
            ("EC_dom", "EC_sub"),
            ("EC_sub", "EC_dom"),
            ("EC_dom", "EC_sub"),
              ("EC_sub", "EC_dom")
             ]
round_count = 0


rounds = [[[1, 2], [4, 3], [6, 5]], #1
        [[1, 5], [4, 2], [6, 3]],   #2
        [[1,3], [4,5], [6, 2]],
        [[1, 2], [4, 3], [6, 5]]
    ]

df_extra = pd.DataFrame()

for (cond_p1, cond_p2) in extra_conflict:
        for (p1, p2) in rounds[round_count]:
            aux_df = pd.DataFrame()
            aux_df["participant"] = [p1, p2]
            aux_df["other_participant"] = [p2, p1]
            aux_df["all_cond"] = [cond_p1, cond_p2]
            aux_df["extra_trials"] = True

            df_extra = pd.concat([df_extra, aux_df])
        round_count +=1
        if round_count > 4:
            round_count=0

df_extra["manip"]          = df_extra.apply(get_manip, axis=1)
df_extra["trial_cond"]     = df_extra.apply(get_condition, axis=1)
df_extra["manip"] =  df_extra.apply(clear_non_conflict_manip, axis=1)

In [157]:
conditions = [("ENC_dom", "ENC_sub"),
            ("ENC_sub", "ENC_dom"),
            ("HNC_dom",  "HNC_sub"),
            ("HNC_sub",  "HNC_dom"),
            ("EC_dom", "EC_sub"),
            ("EC_sub", "EC_dom")
             ]
random.shuffle(conditions)

rounds = [[[1, 2], [4, 3], [6, 5]], #1
        [[1, 3], [5, 2], [4, 6]],   #2
        [[1, 4], [6, 2], [5, 3]],   #3
        [[1, 5], [4, 2], [3, 6]],   #4
        [[1, 6], [3, 2], [4, 5]],   #5
        ]

extra_round_add = [5, 7, 15, 24]

df = pd.DataFrame()
round_count = 0
trial = 1
for i in range(5):
    for (cond_p1, cond_p2) in conditions:

        for (p1, p2) in rounds[round_count]:
            aux_df = pd.DataFrame()
            aux_df["participant"] = [p1, p2]
            aux_df["other_participant"] = [p2, p1]
            aux_df["round_nb"] = [trial, trial]
            aux_df["all_cond"] = [cond_p1, cond_p2]
            aux_df["extra_trials"] = False

            df = pd.concat([df, aux_df])
        trial +=1
        round_count +=1
        if round_count > 4:
            round_count=0

        if trial in extra_round_add:
            df_chunk = df_extra.iloc[:6].copy()
            df_chunk["round_nb"] = trial
            df = pd.concat([df, df_chunk], ignore_index=True)
            df_extra = df_extra.iloc[6:].reset_index(drop=True)
            trial +=1
            
        

df["manip"]          = df.apply(get_manip, axis=1)
df["trial_cond"]     = df.apply(get_condition, axis=1)
df["manip"] =  df.apply(clear_non_conflict_manip, axis=1)

In [151]:
chunk_size = 6

# Extract the first 6 rows
df_chunk = df_extra.iloc[:chunk_size]

# Drop the first 6 rows from df_extra



In [ ]:
add_extra_c_trial = sorted(random.sample(range(1, 31), 4))
print(add_extra_c_trial)

In [314]:
number = 1
while number < 21:
    #Define rounds
    nc_rounds= [[[1, 2], [4, 3], [6, 5]], #1
            [[1, 3], [5, 2], [4, 6]],   #2
            [[1, 4], [6, 2], [5, 3]],   #3
            [[1, 5], [4, 2], [3, 6]],   #4
            [[1, 6], [3, 2], [4, 5]],   #5
            ]

    #Define different conditions
    nc_conditions = [("ENC_dom", "ENC_sub"),
        ("ENC_sub", "ENC_dom"),
        ("HNC_dom",  "HNC_sub"),
        ("HNC_sub",  "HNC_dom")
    ]

    c_conditions = [ ("EC_dom", "EC_sub"),
        ("EC_sub", "EC_dom")
    ]


    #Shuffle conditions
    random.shuffle(nc_conditions)
    random.shuffle(c_conditions)

    #Complete dataframe with all information
    df = pd.DataFrame()
    cpt = 1

    for (cond1, cond2) in nc_conditions:
        for round in nc_rounds:
            for (p1, p2) in round:
                aux_df = pd.DataFrame()
                aux_df["participant"] = [p1, p2]
                aux_df["other_participant"] = [p2, p1]
                aux_df["round_nb"] = [cpt, cpt]
                aux_df["all_cond"] = [cond1, cond2]
                
                df = pd.concat([df, aux_df])
            cpt+=1

    for (cond1, cond2) in c_conditions:
        
        #Conflict rounds = 7
        c_rounds= [[[1, 2], [4, 3], [6, 5]], #1
                [[1, 3], [5, 2], [4, 6]],   #2
                [[1, 4], [6, 2], [5, 3]],   #3
                [[1, 5], [4, 2], [3, 6]],   #4
                [[1, 6], [3, 2], [4, 5]],   #5
                ]
        #add two more rounds, and randomize pairings. 
        others_round_6 = [2,5,6]
        others_round_7 = [2,5,6]
        random.shuffle(others_round_6)
        random.shuffle(others_round_7)

        round_6 = [[1, others_round_6[0]], [3, others_round_6[1]], [4, others_round_6[2]]]
        round_7 = [[1, others_round_7[0]], [3, others_round_7[1]], [4, others_round_7[2]]]

        c_rounds.append(round_6)  # Add round_6 to the list
        c_rounds.append(round_7)  # Add round_7 to the list

        for round in c_rounds:
            for (p1, p2) in round:
                aux_df = pd.DataFrame()
                aux_df["participant"] = [p1, p2]
                aux_df["other_participant"] = [p2, p1]
                aux_df["round_nb"] = [cpt, cpt]
                aux_df["all_cond"] = [cond1, cond2]
                
                df = pd.concat([df, aux_df])
            cpt+=1
            

    #Add useful information
    df["manip"]          = df.apply(get_manip, axis=1)
    df["trial_cond"]     = df.apply(get_condition, axis=1)
    df["manip"] =  df.apply(clear_non_conflict_manip, axis=1)

    #Randomize round numbers
    unique_rounds = df['round_nb'].unique()

    # Shuffle the unique round numbers
    shuffled_rounds = np.random.permutation(unique_rounds)

    # Create a mapping from original rounds to shuffled rounds
    round_mapping = dict(zip(unique_rounds, shuffled_rounds))

    # Apply the mapping to the dataframe
    df['round_nb'] = df['round_nb'].map(round_mapping)

    df = df.sort_values("round_nb")


    players = ["1", "2", "3", "4", "5", "6"]
    df["other_player_str"] = df["other_participant"].astype(str)
    df["participant_str"] = df["participant"].astype(str)

    distance_dict = {}

    for player in players:
        df_player     = df[df["participant_str"] == player]
        other_players = [p for p in players if p != player]

        distance_dict[player] = 0
        for other in other_players:
            
            last_seen = 0
            for index,row in df_player.iterrows():
                if other == row["other_player_str"]:
                    distance  = row["round_nb"] - last_seen
                    last_seen = row["round_nb"]
                    if distance == 1:
                        distance_dict[f"{player}"] += 1
                    
    distance_dict

    if distance_dict["1"] < 3:
        print(distance_dict)
        print(f"found distance less than 3, saving csv number {number}")
        df.to_csv(f"pairing_{number}.csv")

        number += 1

{'1': 2, '2': 2, '3': 4, '4': 3, '5': 4, '6': 3}
found distance less than 4, saving csv number 1
{'1': 2, '2': 3, '3': 2, '4': 4, '5': 2, '6': 3}
found distance less than 4, saving csv number 2
{'1': 2, '2': 2, '3': 4, '4': 5, '5': 6, '6': 3}
found distance less than 4, saving csv number 3
{'1': 2, '2': 4, '3': 3, '4': 5, '5': 3, '6': 3}
found distance less than 4, saving csv number 4
{'1': 2, '2': 3, '3': 3, '4': 2, '5': 2, '6': 2}
found distance less than 4, saving csv number 5
{'1': 2, '2': 2, '3': 4, '4': 2, '5': 4, '6': 2}
found distance less than 4, saving csv number 6
{'1': 2, '2': 2, '3': 5, '4': 3, '5': 5, '6': 3}
found distance less than 4, saving csv number 7
{'1': 1, '2': 1, '3': 3, '4': 2, '5': 3, '6': 2}
found distance less than 4, saving csv number 8
{'1': 2, '2': 2, '3': 2, '4': 3, '5': 3, '6': 2}
found distance less than 4, saving csv number 9
{'1': 1, '2': 1, '3': 3, '4': 3, '5': 4, '6': 2}
found distance less than 4, saving csv number 10
{'1': 2, '2': 3, '3': 3, '4':

In [20]:
#Define rounds
rounds=  [[[1, 2], [4, 3], [6, 5]], #1
        [[1, 3], [5, 2], [4, 6]],   #2
        [[1, 4], [6, 2], [5, 3]],   #3
        [[1, 5], [4, 2], [3, 6]],   #4
        [[1, 6], [3, 2], [4, 5]],   #5
        ]

#Define different conditions
conditions = [ ("EC_dom", "EC_sub"),
    ("EC_sub", "EC_dom"), 
    ("ENC_dom", "ENC_sub"),
    ("ENC_sub", "ENC_dom"),
    ("HNC_dom",  "HNC_sub"),
    ("HNC_sub",  "HNC_dom")
]
number = 1
while number <= 1:

    # Initialize the restart flag
    restart = False

    #Shuffle conditions
    random.shuffle(conditions)
    print(conditions)

    #Complete dataframe with all information
    df = pd.DataFrame()
    cpt = 1
    for (cond1, cond2) in conditions:
        for round in rounds:
            for (p1, p2) in round:
                aux_df = pd.DataFrame()
                aux_df["participant"] = [p1, p2]
                aux_df["other_participant"] = [p2, p1]
                aux_df["round_nb"] = [cpt, cpt]
                aux_df["all_cond"] = [cond1, cond2]
                
                df = pd.concat([df, aux_df])
            cpt+=1
        

    #Add useful information
    df["manip"]          = df.apply(get_manip, axis=1)
    df["trial_cond"]     = df.apply(get_condition, axis=1)
    df["manip"] =  df.apply(clear_non_conflict_manip, axis=1)

    #Randomize round numbers
    unique_rounds = df['round_nb'].unique()

    # Shuffle the unique round numbers
    shuffled_rounds = np.random.permutation(unique_rounds)

    # Create a mapping from original rounds to shuffled rounds
    round_mapping = dict(zip(unique_rounds, shuffled_rounds))

    # Apply the mapping to the dataframe
    df['round_nb'] = df['round_nb'].map(round_mapping)

    df = df.sort_values("round_nb")
       
    number +=1  


#df.to_csv(f"pairing_{number}.csv")
#number+=1

players = ["1", "2", "3", "4", "5", "6"]
df["other_player_str"] = df["other_participant"].astype(str)
df["participant_str"] = df["participant"].astype(str)

distance_dict = {}

for player in players:
    df_player     = df[df["participant_str"] == player]
    other_players = [p for p in players if p != player]

    distance_dict[player] = 0
    for other in other_players:
        
        last_seen = 0
        for index,row in df_player.iterrows():
            if other == row["other_player_str"]:
                distance  = row["round_nb"] - last_seen
                last_seen = row["round_nb"]
                if distance == 1:
                    distance_dict[f"{player}"] += 1
                
distance_dict

#6 times it happens that participants are paired consecutively in the first pilot. 
#the minimum number of times it can happen is 2 per participant. We want to collect these csvs. 

[('ENC_dom', 'ENC_sub'), ('ENC_sub', 'ENC_dom'), ('HNC_dom', 'HNC_sub'), ('EC_dom', 'EC_sub'), ('HNC_sub', 'HNC_dom'), ('EC_sub', 'EC_dom')]


{'1': 2, '2': 2, '3': 2, '4': 2, '5': 2, '6': 2}

In [66]:
players = ["1", "2", "3", "4", "5", "6"]
df["other_player"] = df["other_participant"].astype(str)
df["participant"] = df["participant"].astype(str)

distance_dict = {}

for player in players:
    df_player     = df[df["participant"] == player]
    other_players = [p for p in players if p != player]

    distance_dict[player] = []
    print(distance_dict)

    print("PLAYER", player)

    for other in other_players:
        print("other player:", other)
        last_seen = 0
        for index,row in df_player.iterrows():
            if other == row["other_player"]:
                distance  = row["round_nb"] - last_seen
                last_seen = row["round_nb"]
                if distance == 1:
                    distance_dict[f"{player}"].append(distance)
                print(distance)
distance_dict

{'1': []}
PLAYER 1
other player: 2
1
4
2
3
12
2
other player: 3
2
6
5
4
6
5
other player: 4
4
2
10
2
1
8
other player: 5
9
3
2
6
1
4
other player: 6
3
8
4
11
3
1
{'1': [1, 1, 1, 1], '2': []}
PLAYER 2
other player: 1
1
4
2
3
12
2
other player: 3
3
8
4
11
3
1
other player: 4
9
3
2
6
1
4
other player: 5
2
6
5
4
6
5
other player: 6
4
2
10
2
1
8
{'1': [1, 1, 1, 1], '2': [1, 1, 1, 1], '3': []}
PLAYER 3
other player: 1
2
6
5
4
6
5
other player: 2
3
8
4
11
3
1
other player: 4
1
4
2
3
12
2
other player: 5
4
2
10
2
1
8
other player: 6
9
3
2
6
1
4
{'1': [1, 1, 1, 1], '2': [1, 1, 1, 1], '3': [1, 1, 1, 1], '4': []}
PLAYER 4
other player: 1
4
2
10
2
1
8
other player: 2
9
3
2
6
1
4
other player: 3
1
4
2
3
12
2
other player: 5
3
8
4
11
3
1
other player: 6
2
6
5
4
6
5
{'1': [1, 1, 1, 1], '2': [1, 1, 1, 1], '3': [1, 1, 1, 1], '4': [1, 1, 1, 1], '5': []}
PLAYER 5
other player: 1
9
3
2
6
1
4
other player: 2
2
6
5
4
6
5
other player: 3
4
2
10
2
1
8
other player: 4
3
8
4
11
3
1
other player: 6
1
4
2
3
12
2


In [154]:
df.loc[df["round_nb"] == 1]

,participant,other_participant,round_nb,all_cond,extra_trials,manip,trial_cond
0,1,2,1,ENC_sub,False,submissive,ENC
1,2,1,1,ENC_dom,False,dominant,ENC
0,4,3,1,ENC_sub,False,submissive,ENC
1,3,4,1,ENC_dom,False,dominant,ENC
0,6,5,1,ENC_sub,False,submissive,ENC
1,5,6,1,ENC_dom,False,dominant,ENC


In [156]:
df.groupby(["participant"]).count()[["round_nb"]]

,round_nb
participant,
1,30
2,30
3,30
4,30
5,30
6,30


In [143]:
df_extra.groupby(["participant", "manip"]).count()[["round_nb"]]

round_nb
participant manip               
1           dominant           2
            submissive         2
2           dominant           2
            submissive         2
3           dominant           2
            submissive         2
4           dominant           2
            submissive         2
5           dominant           2
            submissive         2
6           dominant           2
            submissive         2

In [120]:
df.groupby(["participant", "other_participant"]).count()[["round_nb"]]

round_nb
participant other_participant          
1           2                         6
            3                         6
            4                         6
            5                         6
            6                         6
2           1                         6
            3                         6
            4                         6
            5                         6
            6                         6
3           1                         6
            2                         6
            4                         6
            5                         6
            6                         6
4           1                         6
            2                         6
            3                         6
            5                         6
            6                         6
5           1                         6
            2                         6
            3                         6
            4                         6
            6                         6
6           1                         6
            2                         6
            3                         6
            4                         6
            5                         6

In [96]:
df.groupby(["participant", "all_cond"]).count()[["round_nb"]]

round_nb
participant all_cond          
1           EC_dom           7
            EC_sub           7
            ENC_dom          5
            ENC_sub          5
            HNC_dom          5
            HNC_sub          5
2           EC_dom           7
            EC_sub           7
            ENC_dom          5
            ENC_sub          5
            HNC_dom          5
            HNC_sub          5
3           EC_dom           6
            EC_sub           8
            ENC_dom          5
            ENC_sub          5
            HNC_dom          5
            HNC_sub          5
4           EC_dom           6
            EC_sub           8
            ENC_dom          5
            ENC_sub          5
            HNC_dom          5
            HNC_sub          5
5           EC_dom           7
            EC_sub           7
            ENC_dom          5
            ENC_sub          5
            HNC_dom          5
            HNC_sub          5
6           EC_dom           9
            EC_sub           5
            ENC_dom          5
            ENC_sub          5
            HNC_dom          5
            HNC_sub          5

In [97]:
df.groupby(["participant", "trial_cond"]).count()[["round_nb"]]

round_nb
participant trial_cond          
1           EC                14
            ENC               10
            HNC               10
2           EC                14
            ENC               10
            HNC               10
3           EC                14
            ENC               10
            HNC               10
4           EC                14
            ENC               10
            HNC               10
5           EC                14
            ENC               10
            HNC               10
6           EC                14
            ENC               10
            HNC               10

In [98]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df.groupby(["participant", "other_participant", "trial_cond", "manip"]).count()[["round_nb"]])

                                                     round_nb
participant other_participant trial_cond manip               
1           2                 EC         dominant           2
                                         submissive         1
                              ENC        dominant           1
                                         submissive         1
                              HNC        dominant           1
                                         submissive         1
            3                 EC         dominant           1
                                         submissive         2
                              ENC        dominant           1
                                         submissive         1
                              HNC        dominant           1
                                         submissive         1
            4                 EC         dominant           2
                                         submissive         1
        